# Weighted Max-Cut with QAOA

## Brief Problem Description

The problem of interest is the weighted max cut problem. Given a set of vertices and weighted edges connecting some of the vertices, we are interested in separating the vertices into two sets such that the sum of the weights of the edges between the sets is maximized.

In [ ]:
%matplotlib inline 

import time
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
import Qconfig
from tqdm import tqdm
from random import randint, choice, uniform
from math import ceil
from statistics import stdev, mean
from skopt import gbrt_minimize, dummy_minimize, forest_minimize, gp_minimize
from qiskit import register, available_backends, QuantumCircuit, QuantumRegister, \
        ClassicalRegister, execute

register(Qconfig.APItoken, Qconfig.config["url"])
pbar = None
DEBUG = False

def debug(string):
    if DEBUG:
        sys.stdout.write(string)
        sys.stdout.flush()

## Problem Encoding

In order for a problem to be solvable using QAOA, it must satisfy  two main requirements. One must to be able to:
 - Encode a candidate solution to the problem using a string of qubits.
 - Evaluate the 'cost' of a candidate solution with a one or more cost operators.
 
### Candidate Solutions

In the case of Weighted Max Cut, we are a given a graph with N nodes and E edges, and we need to be able to encode a 'cut' of the graph in a bitstring. We are able to accomplish this by giving each node a distinct index from 0 to n-1. Thus, an n-length bitstring can encode a 'cut' to the graph, where one cut contains all the nodes with a 0 at their index into the bitstring, and the other contains nodes with a 1. Below is code containing the logic required to build random problem instances, evaluate candidate bitstring solutions, and determine the maximum possible cost.

In [ ]:
class Graph():
    def __init__(self, N, randomize=True):
        ''' Initialize a random graph with N vertices. '''
        self.N = N
        self.E = 0
        self.adj = {n:dict() for n in range(N)}

        # For storing information about each run.
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []

        # Randomly generate edges.
        if randomize:
            self.randomize()

    def randomize(self):
        ''' Randomly generate edges for this graph. '''

        # Generate list of tuples for all possible directed edges.
        all_possible_edges = set([(x,y) for x in range(self.N) for y in range(self.N) if x != y])

        # Sanity check, ensuring we generated the correct number of edges.
        e_gen = len(all_possible_edges) / 2
        e_shd = self.N * (self.N-1) / 2
        assert e_gen == e_shd , "%d != %d" % (e_gen, e_shd)

        # Choose a random number of edges for this graph to have. 
        # Note, we stop at len/2 because we generated directed edges,
        # so each edge counts twice.
        num_edges = randint(1, len(all_possible_edges)/2)
        for i in range(num_edges):
            # Choose an edge, remove it and its directed complement from the list.
            e = choice(list(all_possible_edges))
            all_possible_edges.remove(e)
            all_possible_edges.remove(e[::-1])

            # Unpack tuple into vertex ints.
            u, v = int(e[0]), int(e[1])

            # Choose a random weight for each edge.
            weight = randint(1, 100)

            #weight = 1
            self.add_edge(u, v, weight)


    def add_edge(self, u, v, weight):
        ''' Add an edge to the graph. '''
        self.E += 1
        self.adj[u][v] = weight

    def get_edges(self):
        ''' Get a list of all edges. '''
        edges = []
        for u in self.adj:
            for v in self.adj[u]:
                edges.append((u, v, self.adj[u][v]))
        return edges

    def get_score(self,bitstring):
        ''' Score a candidate solution. '''
        assert len(bitstring) == self.N

        score = 0

        # For every edge u,v in the graph, add the weight
        # of the edge if u,v belong to different cuts
        # given this candidate solution.

        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    score += self.adj[u][v]
        return score

    def optimal_score(self):
        '''
        Returns (score, solutions) holding the best possible solution to the
        MaxCut problem with this graph.
        '''

        best = 0
        best_val = []

        # Iterate over all possible candidate bitstrings
        # Note: the bitstrings from 0 - N/2 are symmetrically
        # equivalent to those above
        for i in range(ceil((2 ** self.N)/2)):
            # Convert number to 0-padded bitstring.
            bitstring = bin(i)[2:]
            bitstring = (self.N - len(bitstring)) * "0" + bitstring

            sc = self.get_score(bitstring)
            if sc > best:
                best = sc
                best_val = [bitstring]
            elif sc == best:
                best_val.append(bitstring)
        return best, best_val

    def update_score(self, bitstring):
        ''' Scores the given bitstring and keeps track of best. '''
        score = self.get_score(bitstring)
        if score > self.currentScore:
            self.currentScore = score
            self.currentBest = bitstring
        return score
    
    def clear_runs(self):
        ''' Clear data from past runs. '''
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []
        
    def add_run(self, gamma, beta, expected_value):
        ''' Save the data from each run iteration. '''
        self.runs.append([gamma, beta, expected_value])
        
    def __str__(self):
        return "Graph with %d vertices %d edges.\nAdjacency List: %s" % (self.N, self.E, self.adj)

# Sample graph.
g = Graph(5)
print(g)

### Cost Function

We can evaluate a candidate solution by applying a 'cost' operator to each pair of nodes that share an edge. In particular, for every edge (u,v) in the graph, we apply a cost operator that:
  - Applies a phase of $e^{i w \gamma}$, the weight of the edge, to both qubits encoding those nodes if they belong to different cuts.
  - Does nothing if they belong to the same cut.
  
The cost operator shown below implements the above constraints:
![Cost Operator](img/cost.gif)


This cost operator can be equivalently represented by $\frac{w}{2}(I-Z_1Z_2)$.

## QAOA Description

QAOA works in two stages: a quantum computing stage and a classical stage.

### Quantum Stage:

The quantum stage takes in two parameters, gamma and beta, and a tuning hyperparameter p. In general, increasing p is guaranteed not to make your solution worse, though there may be diminishing returns after a certain point. For our application, we chose a p of 1, as even with p=1 we were consistently able to find the optimal solution on the problem sizes we could simulate.

First, n qubits are prepared in a superposition over all candidate solutions. Gates are then applied like so: $W_pV_p$ ... $W_1V_1$ |$\Phi$>, where W and V refer to Hamiltonians encoding the cost and driver functions (defined in detail below). To calculate the expected value, we measure the input state several times, getting several candidate solutions along with the amount of times they occur. For each candidate solution, we evaulate it's cost and multiply it by the probability of that solution occuring (where probability is the number of times we measured it divided by the total number of shots). This is generalized below:
```
EXP = P(state ‘000..0’) * cost(‘000..0’) + … + P(state ‘111..1’) * cost(‘111..1’)
```

### Classical Stage:

In the classical stage, two parameters gamma and beta are randomly chosen and fed into the quantum stage, getting an expectation value. We then run a classical optimizer over gamma and beta, using the quantum stage as our 'black-box' cost function, until we are satsified with the expectation value.

### Cost and Driver Hamiltonians C and B:

The cost hamiltonian V can be expressed by $e^{-i \gamma C}$, where C is a global cost operator. In the case of Weighted Max Cut, we can express the cost operator as a sum of the local cost operators, which act on pairs of qubits sharing an edge, defined above. For each of the qubits corresponding to the vertices of that edge, we apply a phase of $e^{i w \gamma}$, where $w$ is the weight of the edge between those two qubits.

The driver hamiltonian W can be expressed by $e^{-i \beta B}$, where B is the an operator that flips all the input qubits. 

Below is some code that, given $\beta, \gamma$, prepares and evaluates the "Quantum Stage" of the algorithm described above, returning an expectation value.

In [ ]:
def get_expectation(x, g, NUM_SHOTS=1024):
    # Look for progress bar as a global variable.
    global pbar
    
    gamma, beta = x

    debug("Cost of Gamma: %s, beta: %s... " % (gamma, beta))

    # Construct quantum circuit.
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)

    # Apply hadamard to all inputs.
    for i in range(g.N):
        qc.h(q[i])

    # Apply V for all edges.
    for edge in g.get_edges():
        u, v, w = edge

        # Apply CNots.
        qc.cx(q[u], q[v])

        qc.u1(gamma*w, q[v])

        # Apply CNots.
        qc.cx(q[u], q[v])

    # Apply W to all vertices.
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])


    # Measure the qubits.
    for i in range(g.N):
        qc.measure(q[i], c[i])

    # Run the simluator.
    job = execute(qc, backend='ibmq_qasm_simulator', shots=NUM_SHOTS)
    results = job.result()
    result_dict = results.get_counts(qc)

    debug("done!\n")

    # Calculate the expected value of the candidate bitstrings.
    exp = 0
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / NUM_SHOTS
        score = g.update_score(bitstring)

        # Expected value is the score of each bitstring times
        # probability of it occuring.
        exp += score * prob

    debug("\tExpected Value: %s\n" % (exp))
    debug("\tBest Found Solution: %s, %s\n" % (g.currentScore, g.currentBest))

    g.add_run(gamma, beta, exp)

    # Try updating progress bar if defined.
    try:
        pbar.update(1)
    except:
        pass
    
    return exp # bc we want to minimize


## Gamma/Beta vs. Expectation

Before choosing a classical optimizer, we first wanted to plot the gradients of both gamma and beta to see what types of gradients we were attempting to optimize over. Below is the code we used and some graphs of the gradients of both gamma and beta with all other variables held constant.

In [ ]:
def hold_constant(vary="beta"):
    ''' Plots expected value vs. gamma/beta, holding the rest of the variables constant.'''
    # Choose some random starting beta/gamma and graph.
    lim = np.pi if vary == "gamma" else 2*np.pi
    constant_var = uniform(0, lim)
    g = Graph(5)

    # RUNS # of runs at each gamma for error bars.
    RUNS = 3

    # Keep track of gammas, expected values, for plotting.
    pts, exp, std = [], [], []

    # The maximum possible expected value is the maximum possible weighted cut.
    opt = g.optimal_score()[0]
    debug("Optimal score: %s\n" % (opt))
    
    # Number of data points to collect.
    NUM_RUNS = 3
    MIN = 0
    MAX = 2*np.pi if vary == "gamma" else np.pi
    
    # For progress bar.
    global pbar
    pbar = tqdm(total=NUM_RUNS*RUNS)
    
    points = np.linspace(MIN, MAX, NUM_RUNS)
    for point in points:
        pts.append(point)

        # Calculate expected values.
        vals = []
        for i in range(RUNS):
        
            # Params are passed in as gamma, beta, so order matters.
            params = [point, constant_var] if vary == "gamma" else [constant_var, point]
            vals.append(get_expectation(params, g))

        # Calculate mean, standard deviation.
        exp.append(mean(vals))
        std.append(stdev(vals))


    fig, ax = plt.subplots()

    ax.errorbar(x=pts, y=exp, yerr=std, fmt='o-', markersize=10)
    ax.legend(loc=2)

    # Names for plotting.
    vary_name = "Gamma" if vary == "gamma" else "Beta"
    const_name = "Beta" if vary_name == "Gamma" else "Gamma"
    
    ax.set_title("Effect of Varying %s with %s = %s" % (vary_name, const_name, constant_var))
    ax.set_xlabel("%s" % (vary_name)) 
    ax.set_ylabel("Expected Value")


    plt.show()
#hold_constant()

![gamma vs. exp](img/gamma_change.png)
![beta vs.exp](img/beta_change.png)

Not bad! The above plots look pretty promising, and seem to imply that a gradient-based optimizer should be able to find local maxima nicely. **However**, these graphs are misleading. It actually took us a good three or four runs to get graphs with a smooth gradient. Most of the runs looked more like:

![Bad Gamma](img/gamma_weird.png)

## Evaluating Gradients

From creating several plots, we learned that the gradients were very noisy with respect to other constant variables. For example, changing the constant value of beta slightly would produce an entirely different gradient for gamma. Even without changing the constant variables, the gradients would be very different across runs! 

An interesting note is that the gradient of beta was consistently smoother than that of gamma. Though we chose not to explore it, it could be possible to choose gamma randomly and run gradient-based optimization over just beta.

### Choosing Classical Optimizers

We evaluated several different classical optimizers. We originally started with "gradient descent" optimizers, which work by, given starting beta/gamma values, perturb them slightly in order to approximate the local gradient and then jump. However, we noticed that the jump seemed to increase/decrease the expectation value with equal probability. In other words, the local gradient approximation was not very accurate. This is likely due to the large amount of noise present in the gradients: slighting perturbing the values of beta/gamma, like we found earlier, could result in huge changes in expected value.

Another problem with gradient descent optimizers is that they, in general, require many function evaluations at each step. The Scikit-learn optimizer we were initially using used four different function evaluations to calculate the gradient (likely associated with increasing/decreasing beta/gamma). In our case, where function evaluations are so expensive, this overhead was too much.

#### Scikit-Optimize

We eventually stumbled across [Scikit-Optimize](https://scikit-optimize.github.io/), a set of optimization functions built specifically to minimize calls to expensive black-box functions. Below is a brief description of the four we evaluated:
  - *Forest_Minimize* - Uses sequential decision trees.
  - *Gbrt_Minimize*   - Uses Gradient Boosted decision trees.
  - *Gp_Minimize*     - Uses good 'ole Baynesian regression.
  - *Dummy_Minimize*  - Randomly chooses points from the sample space.


In [ ]:
# Plot different types of optimizers.
def compare_optimizers(num_instances=4, graph_size=15, n_calls=8, n_random_starts=2):
    global pbar
    pbar = None
    
    # For progress bar.
    pbar = tqdm(total=num_instances*n_calls*4)
    
    instances = [Graph(graph_size) for _ in range(num_instances)]
    
    # Percent of optimal score acheived by each algorithm.
    dummy = []
    decision_trees = []
    gradient_boosted_trees = []
    baynesian = []
    
    # For each instance, run each algorithm.
    for inst in instances:
        # Scikit functions only take in parameters and want to minimize values.
        # Create a wrapper function to format get_expectation.
        sk_get_exp = lambda x: -1*get_expectation(x, inst)

        
        opt = inst.optimal_score()[0]
        
        # Dummy.
        inst.clear_runs()
        dummy_minimize(func=sk_get_exp,
                      dimensions=[(0,2*np.pi),(0,np.pi)],
                      n_calls=n_calls)
        dummy.append(float(inst.currentScore) / opt)

        # Decision Trees.
        inst.clear_runs()
        forest_minimize(func=sk_get_exp,
                      dimensions=[(0,2*np.pi),(0,np.pi)],
                      n_calls=n_calls,
                      n_random_starts=n_random_starts)
        decision_trees.append(float(inst.currentScore) / opt)
        
        # Gradient Boosted Decision Trees.
        inst.clear_runs()
        gbrt_minimize(func=sk_get_exp,
                      dimensions=[(0,2*np.pi),(0,np.pi)],
                      n_calls=n_calls,
                      n_random_starts=n_random_starts)
        gradient_boosted_trees.append(float(inst.currentScore) / opt)
        
        # Baynesian.
        inst.clear_runs()
        gp_minimize(func=sk_get_exp,
                      dimensions=[(0,2*np.pi),(0,np.pi)],
                      n_calls=n_calls,
                      n_random_starts=n_random_starts)
        baynesian.append(float(inst.currentScore) / opt)

    # Compare mean/stdev of % opt. achieved for each algorithm.
    print("-- % of Optimal Achieved, Mean and Std. Dev --")
    print("Random Sampling:\nMean: %s\nStd. Dev: %s" % (mean(dummy), stdev(dummy)))
    print("Decision Trees:\nMean: %s\nStd. Dev: %s" % (mean(decision_trees), stdev(decision_trees)))
    print("Gradient Boosted Decision Trees:\nMean: %s\nStd. Dev: %s" % (mean(gradient_boosted_trees), stdev(gradient_boosted_trees)))
    print("Baynesian Optimization:\nMean: %s\nStd. Dev: %s" % (mean(baynesian), stdev(baynesian)))
    
#compare_optimizers()

### Optimizer Results

```
-- % of Optimal Achieved, Mean and Std. Dev --
Random Sampling:
    Mean: 0.9918135656630878
    Std. Dev: 0.008893516482779347
Decision Trees:
    Mean: 0.9911853052336336
    Std. Dev: 0.013767350119746134
Gradient Boosted Decision Trees:
    Mean: 0.9874128354367063
    Std. Dev: 0.01234703421622558
Baynesian Optimization:
    Mean: 0.9976966107272129
    Std. Dev: 0.00398958725007638
```

Suprisingly, all four optimization methods pretty equally well, including random sampling! The fact the all of them achieved so close to optimal seems to suggest that QAOA is pretty efficient at find decent solutions, even with random beta and gamma values and p=1 (at least for this problem size). In addition, our results showed the Baynesian regression was the most accurate and precise optimizer. Though we aren't extremely certain why this is the case, we think it has to do with large amounts of noise.

Given that our gradients had small general trends and lots of noise, it makes sense that a normal regression can work well, because as long as their are as many points above as below the true trend line, our approximate trend line should be pretty accurate. Furthermore, regression only needs a couple of points to create a decent trend line. The fact that we were limiting the number of function calls to 8 could be another reason why regression outperformed other optimization algorithms.

## Further Results and Discussion

### Minimum, Maximum and Mean Costs across Problem Instances

After deciding on Baynesian regression as our optimizer, we evaluated our algorithm on several instances of the problem. Below is the code and our plot of expected value and the best score achieved, relative to the maximum possible score.

In [ ]:
def instance_cost(num_instances=25, num_vert=15, num_runs=4):
    '''
    For several random problem instances, plot the cost of the output state.
    Plot average, maximum and minimum cost.
    '''

    # Prepare several random instances of the problem.
    instances = [Graph(num_vert) for _ in range(num_instances)]

    # For progress bar.    
    global pbar
    pbar = tqdm(total=num_instances*num_runs)

    # For holding iteration number and expected values.
    its, lows, averages, highs, best_founds = [], [], [], [], []

    it = 1
    # Calculate expected values using Baynesian Regression.
    for graph in instances:
        
        # Run the optimizer.
        sk_get_exp = lambda x: -1*get_expectation(x, graph)
        gp_minimize(func=sk_get_exp,
                      dimensions=[(0,2*np.pi),(0,np.pi)],
                      n_calls=num_runs,
                      n_random_starts=max(1, int(num_runs*0.2)))
        
        # Save results.
        its.append(it)
        curr_opt = graph.optimal_score()[0]
        
        # Min/Avg/Max expected values.
        exps = [r[2] for r in graph.runs] # extract expected values
        lows.append(float(min(exps)) / curr_opt)
        averages.append(float(mean(exps)) / curr_opt)
        highs.append(float(max(exps)) / curr_opt)

        
        # Best found result.

        best_founds.append(float(graph.currentScore) / curr_opt)
        
        it += 1

    plt.title("Costs of Random Instances")
    plt.xlabel("Instance Num")
    plt.ylabel("% Optimal Achieved")


    plt.plot(its, averages, color='blue', label='Average Cost %')
    plt.plot(its, lows, color='green', label='Minimum Cost %')
    plt.plot(its, highs, color='orange', label='Maximum Cost %')
    plt.plot(its, best_founds, color='red', label='Best Found Cost %')

    plt.legend()

    plt.savefig("img/avg_cost.png")
#instance_cost()

![Cost across Instances](img/avg_cost_good-magic.png)

The above plot shows that, for problems of size N=15, QAOA is pretty effective at finding close to the optimal solution. It also shows that expected value is very precise, with the min/avg/max lines all basically coinciding with each other. Finally, it shows that expected value is not always an accurate indicator of the best score found. In the case of instance 16, the average expected value was less than 50% of optimal, and yet there was no noticeable dip in the best cost found. However, instance 22's dip in expected value does correlate with a dip in best found cost.

## Conclusion

We have no guarantee that QAOA will give us a decent solution, and with p=1, we have even less of a guarantee. However, it seems that QAOA manages to consistently provide solutions within 90% of optimal (at least for our problem size). One thing we noticed when watching the optimizer work, is that the first iteration tended to always start within 80% of the optimal solution, even when we fed in random beta/gamma starting values. It would be interesting to see how well a random candidate bitstring would do compared to random beta/gamma values -- my intuition is that the starting point that QAOA is able to achieve is much better on average than choosing a random candidate solution would be classically. 

Overall, we were relatively surprised by the performance of QAOA, with and without classical optimization, and we think the results definitely merit more looking into.